# Comp 561 Final Project

## Import Statements

First, we need to get our data, so figure out how to install it from 10x genomics. The data consist of 3k PBMCs from a Healthy Donor and are freely available from 10x Genomics (here from this webpage). On a unix system, you can uncomment and run the following to download and unpack the data. The last line creates a directory for writing processed data.

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc